# **WOD to CSIRO Ocean Data Archive**

This notebook deomstrates the proceedure for converting World Ocean Database (WOD) profiles in ragged array format to 
the CSIRO Ocean Data Archive (CODA) format. The basic steps are: 
- Download data from the Amazon Web Service (AWS) s3 bucket to a local machine;
- Add a "cast index" to the ragged array data to facilitate work on individual casts (also known as "profiles");
- Convert to CODA format and save to local disk

The local CODA format files are organised by observational platform (CTD, XBT, glider, profiling float, moored instrument, etc...), year and date. An individual file for each date, variable, and observing platform is produced to enable easy ingestion into data assimilation systems.



In [ ]:
import xarray
import fsspec
import numpy as np
import s3fs
import proplot
import os

from matplotlib import pyplot as plt
import pandas

In [6]:
def create_profile_index(wod_dataset,variable_name):
    
    
    if variable_name in wod_dataset.data_vars or variable_name=='z':
        
        cast_row_size = wod_dataset[variable_name + '_row_size'].values 
        cast_counter = cast_row_size.cumsum().astype(int)
    
        cast_counter = np.concatenate([np.zeros([1],dtype=int),cast_counter])
        cast_index   = xarray.full_like(wod_dataset[variable_name + '_obs'],fill_value=0,dtype=int)
    
        cast_row_counter = 0
        for i_cast in range(0,wod_dataset['casts'].size):

            current_cast_row_size = cast_row_size[i_cast] #.astype(int)
            if not np.isnan(current_cast_row_size):
            
                cast_index[cast_row_counter:cast_row_counter+int(current_cast_row_size)] =  wod_dataset['casts'][i_cast].astype(int)
                cast_row_counter = cast_row_counter + int(current_cast_row_size)
       
        return cast_index

    else:
        return None

**Initiate the time period to interogate**\
Set the start and the end year (inclusive)

In [8]:
START_YEAR = 2010
END_YEAR   = 2011

**Download the data from the AWS S3 bucket**\
Requires the use of the s3 package. Download speeds are a bit slow, so this step could be improved.

In [9]:
obs_type = ['ctd','xbt','pfl'] #,'mrb','gld'] #,'drb'] #,'gld']
#obs_type = ['ctd']

base_s3_url = 's3://noaa-wod-pds'

data_container = {}

output_directory = '/tube1/cha674/CARS_2022_ancillary_data/WOD_2018'

for i_year in range(START_YEAR,END_YEAR+1):

    data_container_for_year = {}
    for i_obs_type in obs_type:
        url = base_s3_url + '/' + str(i_year) + '/wod_' + i_obs_type + '_' + str(i_year) + '.nc' #_pfl_2018.nc'
        print(url)
        fs = s3fs.S3FileSystem(anon=True)
        file_obj = fs.open(url)
        ds = xarray.open_dataset(file_obj,engine='h5netcdf')
        data_container_for_year[i_obs_type] = ds
        
        #Write data out
        if not os.path.isdir(os.path.join(output_directory,str(i_year))):
            os.mkdir(os.path.join(output_directory,str(i_year)))
        print(os.path.join(output_directory,str(i_year),'wod_' + i_obs_type + '_' + str(i_year) + '.nc'))
        ds.to_netcdf(os.path.join(output_directory,str(i_year),'wod_' + i_obs_type + '_' + str(i_year) + '.nc'))
        
    data_container[i_year] = data_container_for_year

s3://noaa-wod-pds/2010/wod_ctd_2010.nc
/tube1/cha674/CARS_2022_ancillary_data/WOD_2018/2010/wod_ctd_2010.nc
s3://noaa-wod-pds/2010/wod_xbt_2010.nc
/tube1/cha674/CARS_2022_ancillary_data/WOD_2018/2010/wod_xbt_2010.nc
s3://noaa-wod-pds/2010/wod_pfl_2010.nc
/tube1/cha674/CARS_2022_ancillary_data/WOD_2018/2010/wod_pfl_2010.nc
s3://noaa-wod-pds/2011/wod_ctd_2011.nc
/tube1/cha674/CARS_2022_ancillary_data/WOD_2018/2011/wod_ctd_2011.nc
s3://noaa-wod-pds/2011/wod_xbt_2011.nc
/tube1/cha674/CARS_2022_ancillary_data/WOD_2018/2011/wod_xbt_2011.nc
s3://noaa-wod-pds/2011/wod_pfl_2011.nc
/tube1/cha674/CARS_2022_ancillary_data/WOD_2018/2011/wod_pfl_2011.nc


**Read the WOD ragged array files from the local archive**\
It is better to download from AWS and then read locally, as performing operations on the remote dataset\
can be quite slow

In [10]:
WOD_directory = '/tube1/cha674/CARS_2022_ancillary_data/WOD_2018'
obs_type = ['ctd','xbt','pfl'] #,'mrb','gld'] #,'drb'] #,'gld']
data_container = {}

for i_year in range(START_YEAR,END_YEAR+1):
    data_container_for_year = {}
    for i_obs_type in obs_type:
        file_to_get = os.path.join(WOD_directory,str(i_year),'wod_' + i_obs_type + '_' + str(i_year) + '.nc')

        ds = xarray.open_dataset(file_to_get)  #,engine='h5netcdf')
        data_container_for_year[i_obs_type] = ds
        
        
    data_container[i_year] = data_container_for_year

**Add the cast index to the WOD data container**\
WOD ragged array format is organised as follows:
- Individual casts each with a "row size" variable
- A (potentially long) data array of all individual cast data values concatenated together

For each data variable (temperature, salinity, pressure, etc...) we add a simple, unique "cast index". 
This step is relatively slow and could probably be multi-processed, as each cast is treated separately

In [11]:
variable_names = ['Temperature','Pressure','Salinity','Oxygen'] #Oxygen
WOD_directory = '/tube1/cha674/CARS_2022_ancillary_data/WOD_2018'


for i_year in range(START_YEAR,END_YEAR+1):
    year_index = i_year-START_YEAR
    print('Working on year: ', i_year)
    for i_obs_type in obs_type:

        print('Working on observational platform: ', i_obs_type)
        for i_var in variable_names:
            print('Working on variable: ', i_var)
            cast_temperature_index = create_profile_index(data_container[i_year][i_obs_type],i_var)
            
            data_container[i_year][i_obs_type][i_var + '_cast_index']    = cast_temperature_index
            
        cast_depth_index       = create_profile_index(data_container[i_year][i_obs_type],'z')
        data_container[i_year][i_obs_type]['z_cast_index']           = cast_depth_index
        

Working on year:  2010
Working on observational platform:  ctd
Working on variable:  Temperature
Working on variable:  Pressure
Working on variable:  Salinity
Working on variable:  Oxygen
Working on observational platform:  xbt
Working on variable:  Temperature
Working on variable:  Pressure
Working on variable:  Salinity
Working on variable:  Oxygen
Working on observational platform:  pfl
Working on variable:  Temperature
Working on variable:  Pressure
Working on variable:  Salinity
Working on variable:  Oxygen
Working on year:  2011
Working on observational platform:  ctd
Working on variable:  Temperature
Working on variable:  Pressure
Working on variable:  Salinity
Working on variable:  Oxygen
Working on observational platform:  xbt
Working on variable:  Temperature
Working on variable:  Pressure
Working on variable:  Salinity
Working on variable:  Oxygen
Working on observational platform:  pfl
Working on variable:  Temperature
Working on variable:  Pressure
Working on variable:  Sa

In [ ]:
CODA_form_data_container = {}

obs_type = ['ctd','xbt','pfl']
for i_year in range(START_YEAR,END_YEAR+1):
    year_index = i_year-START_YEAR
    print(i_year)
    CODA_form_data_container_for_year = {}

    
    for i_obs_type in obs_type:
        print(i_obs_type)
        
        CODA_form_data_container_for_obs_type = {}
        for i_var in variable_names:
            print('variable:', i_var)
            if i_var in data_container[i_year][i_obs_type].data_vars or i_var=='z':
                
                #Get the cast indicies for the current variable
                data_cast_index       = data_container[i_year][i_obs_type][i_var + '_cast_index']
                unique_cast_identifer = np.unique(data_cast_index)

                #Intialize the maximum length
                max_length_profile = 0
            
                #Test to find the max length of any individual profile.
                #NOTE max length and max depth are not the same, particularly
                #for hydrographic (bottle) data
                for i_cast in unique_cast_identifer:
                
                    #Get length of current profile
                    length_profile = (data_cast_index.values==i_cast).sum()
                
                    #is current length greater than the max length?
                    if length_profile>max_length_profile:
                        max_length_profile = length_profile
            
                #END for i_cast
                
                #Initialize CODA data arrays
                data_array  = np.zeros([unique_cast_identifer.size,max_length_profile],dtype='float32')
                depth_array = np.zeros([unique_cast_identifer.size,max_length_profile],dtype='float32')
                depth_array = np.zeros([unique_cast_identifer.size,max_length_profile],dtype='float32')

                data_WOD_flag_array  = np.zeros([unique_cast_identifer.size,max_length_profile],dtype='int8')
                depth_WOD_flag_array = np.zeros([unique_cast_identifer.size,max_length_profile],dtype='int8')

                
                cast_ID         = np.zeros([unique_cast_identifer.size],dtype='int32')
                latitude_array  = np.zeros([unique_cast_identifer.size],dtype='float32')
                longitude_array = np.zeros([unique_cast_identifer.size],dtype='float32')
                time_array      = np.zeros([unique_cast_identifer.size],dtype='datetime64[ns]')
                

        
                #Assign 
                for i_cast in range(0,len(unique_cast_identifer)):
                    i_cast_index = np.nonzero(data_cast_index.values==unique_cast_identifer[i_cast])[0]
                    
                    depth_obs    =  data_container[i_year][i_obs_type]['z'][i_cast_index]
                    data_obs     =  data_container[i_year][i_obs_type][i_var][i_cast_index]
                    
                    if 'z_WODflag' in data_container[i_year][i_obs_type].data_vars:
                        depth_flag   = data_container[i_year][i_obs_type]['z_WODflag'][i_cast_index]
                        depth_WOD_flag_array[i_cast,0:depth_obs.size] = depth_flag
                    if i_var + '_WODflag' in data_container[i_year][i_obs_type].data_vars:
                        data_flag    = data_container[i_year][i_obs_type][i_var + '_WODflag'][i_cast_index]
                        data_WOD_flag_array[i_cast,0:data_obs.size]   = data_flag
                     #Temperature_WODflag

                    
                    data_array[i_cast,0:data_obs.size]   = data_obs
                    depth_array[i_cast,0:depth_obs.size] = depth_obs

                    data_array[i_cast,data_obs.size::]   = np.nan
                    depth_array[i_cast,depth_obs.size::] = np.nan

                
                    cast_ID[i_cast]          = data_container[i_year][i_obs_type]['wod_unique_cast'][unique_cast_identifer[i_cast]]
                    latitude_array[i_cast]   = data_container[i_year][i_obs_type]['lat'][unique_cast_identifer[i_cast]]
                    longitude_array[i_cast]  = data_container[i_year][i_obs_type]['lon'][unique_cast_identifer[i_cast]]
                    time_array[i_cast]       = data_container[i_year][i_obs_type]['time'][unique_cast_identifer[i_cast]].values
                    
                #END for i_cast
                
                CODA_form_data_container_for_obs_type[i_var + '_data']            = data_array
                CODA_form_data_container_for_obs_type[i_var + '_depth']           = depth_array
                CODA_form_data_container_for_obs_type[i_var + '_data_WODflag']    = data_WOD_flag_array
                CODA_form_data_container_for_obs_type[i_var + '_depth_WOD_flag']  = depth_WOD_flag_array
            
                CODA_form_data_container_for_obs_type[i_var + '_lat']             = latitude_array
                CODA_form_data_container_for_obs_type[i_var + '_lon']             = longitude_array
                CODA_form_data_container_for_obs_type[i_var + '_time']            = time_array
                CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast'] = cast_ID

            else:
                print(i_var, ' not found, skipping')
                continue 
            #END     
            
            #CODA_form_data_container_for_obs_type[i_var + '_depth_WOD_flag'] = depth_WOD_flag_array

        #END i_var
        CODA_form_data_container_for_year[i_obs_type] = CODA_form_data_container_for_obs_type
        
        
    #END i_obs
    
    CODA_form_data_container[i_year] = CODA_form_data_container_for_year
#END i_year

2010
ctd
variable: Temperature
variable: Pressure


In [28]:
for i_year in range(START_YEAR,END_YEAR+1):
    year_index = i_year-START_YEAR
    print(i_year)
    CODA_form_data_container_for_year = CODA_form_data_container[i_year]
        
    
    
    for i_obs_type in obs_type:
        
        print(i_obs_type)
        
        #output_file_path_current_year = os.path.join(output_file_path,str(i_year),i_obs_type)

        #if not os.path.isdir(output_file_path_current_year):
        #    os.makedirs(output_file_path_current_year)

        
        CODA_form_data_container_for_obs_type = CODA_form_data_container_for_year[i_obs_type]
        dsa
        
        

2010
ctd


NameError: name 'dsa' is not defined

In [35]:
CODA_form_data_container[2010]['xbt']['Temperature_depth'].shape

(21174, 3561)

In [27]:
obs_type = ['ctd','xbt','pfl']

output_file_path = '/oa-decadal-climate/work/observations/CARSv2_ancillary/CODA/WOD'
for i_year in range(START_YEAR,END_YEAR+1):
    year_index = i_year-START_YEAR
    print(i_year)
    CODA_form_data_container_for_year = CODA_form_data_container[i_year]
        
    
    
    for i_obs_type in obs_type:
        
        print(i_obs_type)
        
        output_file_path_current_year = os.path.join(output_file_path,str(i_year),i_obs_type)

        if not os.path.isdir(output_file_path_current_year):
            os.makedirs(output_file_path_current_year)

        
        CODA_form_data_container_for_obs_type = CODA_form_data_container_for_year[i_obs_type]
        
        
        for i_var in variable_names:
            if i_var + '_data' in CODA_form_data_container_for_obs_type.keys() or i_var=='z':
                print(i_var)
                
                
                #print(CODA_form_data_container_for_obs_type[ i_var + '_data'])
                
                CODA_output_dataset = xarray.DataArray(CODA_form_data_container_for_obs_type[ i_var + '_data'],
                                                     dims=['cast','depth_index'],
                                                     coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast'],
                                                             'depth_index':np.arange(0,CODA_form_data_container_for_obs_type[ i_var + '_data'].shape[1],1)}).to_dataset(name=i_var + '_data')
                CODA_output_dataset[i_var + '_depth'] = xarray.DataArray(CODA_form_data_container_for_obs_type[i_var + '_depth'],
                                                     dims=['cast','depth_index'],
                                                     coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast'],
                                                             'depth_index':np.arange(0,CODA_form_data_container_for_obs_type[ i_var + '_data'].shape[1],1)})
                CODA_output_dataset[i_var + '_data_WODflag'] = xarray.DataArray(CODA_form_data_container_for_obs_type[i_var + '_data_WODflag'],
                                                     dims=['cast','depth_index'],
                                                     coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast'],
                                                             'depth_index':np.arange(0,CODA_form_data_container_for_obs_type[ i_var + '_data'].shape[1],1)})
                CODA_output_dataset[i_var + '_depth_WOD_flag'] = xarray.DataArray(CODA_form_data_container_for_obs_type[i_var + '_depth_WOD_flag'],
                                                     dims=['cast','depth_index'],
                                                     coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast'],
                                                             'depth_index':np.arange(0,CODA_form_data_container_for_obs_type[ i_var + '_data'].shape[1],1)})

                CODA_output_dataset['lat'] = xarray.DataArray(CODA_form_data_container_for_obs_type[i_var + '_lat'],
                                                     dims=['cast'],coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast']})
                CODA_output_dataset['lon'] = xarray.DataArray(CODA_form_data_container_for_obs_type[i_var + '_lon'],
                                                            dims=['cast'],coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast']})
                CODA_output_dataset['time'] = xarray.DataArray(CODA_form_data_container_for_obs_type[i_var + '_time'],
                                                            dims=['cast'],coords={'cast':CODA_form_data_container_for_obs_type[i_var + '_wod_unique_cast']})
                
                CODA_output_dataset['time'].encoding['units'] = 'seconds since 1980-01-01'

                
                
                daily_date_range = np.arange(np.datetime64(str(i_year) + '-01-01'),np.datetime64(str(i_year+1) + '-01-02'),np.timedelta64(1,'D'))
              
                for i_date in range(0,len(daily_date_range)-1):
                    current_date = daily_date_range[i_date] 
                    next_date    = daily_date_range[i_date+1] #.astype('str')
                    
                    
                    idx_time = np.nonzero(np.logical_and( CODA_output_dataset['time'].values>=current_date,
                                                          CODA_output_dataset['time'].values <next_date     ))[0]
                    #date_string = current_date.dt.strftime('%m/%d/%Y')
                    #print(current_date.astype('str'))
                    
                    daily_output_file_name = 'CODA_WOD_' + i_obs_type + '_' + i_var + '_' + current_date.astype('str') + '.nc'
                    
                    daily_CODA_output_dataset = CODA_output_dataset.isel(cast=idx_time)
                    
                    
                    
                    
                
                    #print('Writing output:', os.path.join(output_file_path_current_year,daily_output_file_name))
                
                    daily_CODA_output_dataset.to_netcdf(os.path.join(output_file_path_current_year,daily_output_file_name))
            

2010
ctd
Temperature
Pressure
Salinity
Oxygen
xbt
Temperature
pfl
Temperature
Pressure
Salinity
Oxygen
2011
ctd
Temperature
Pressure
Salinity
Oxygen
xbt
Temperature
pfl
Temperature
Pressure
Salinity
Oxygen


In [25]:
daily_date_range

array(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
       '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
       '2011-01-09', '2011-01-10', '2011-01-11', '2011-01-12',
       '2011-01-13', '2011-01-14', '2011-01-15', '2011-01-16',
       '2011-01-17', '2011-01-18', '2011-01-19', '2011-01-20',
       '2011-01-21', '2011-01-22', '2011-01-23', '2011-01-24',
       '2011-01-25', '2011-01-26', '2011-01-27', '2011-01-28',
       '2011-01-29', '2011-01-30', '2011-01-31', '2011-02-01',
       '2011-02-02', '2011-02-03', '2011-02-04', '2011-02-05',
       '2011-02-06', '2011-02-07', '2011-02-08', '2011-02-09',
       '2011-02-10', '2011-02-11', '2011-02-12', '2011-02-13',
       '2011-02-14', '2011-02-15', '2011-02-16', '2011-02-17',
       '2011-02-18', '2011-02-19', '2011-02-20', '2011-02-21',
       '2011-02-22', '2011-02-23', '2011-02-24', '2011-02-25',
       '2011-02-26', '2011-02-27', '2011-02-28', '2011-03-01',
       '2011-03-02', '2011-03-03', '2011-03-04', '2011-